# Mini-BMN Notebook

Expand mini-BMN hamiltonian into Pauli strings

In [1]:
import qiskit
import numpy as np
import sympy as sp
from collections import Counter
from qiskit.circuit import Parameter, ParameterVector
from qiskit import opflow

In [7]:
class SpecialUnitaryGroup():
    '''
    https://en.wikipedia.org/wiki/Structure_constants
    https://arxiv.org/pdf/2108.07219.pdf
    '''
    def __init__(self, N):
        self.N = N
        self.structure_constants = self.generate_structure_constants()
        self.generators = self.generate_generators()
        self.check()

    def alpha(self, n, m):
        assert (1 <= m) and (m < n) and (n <= self.N)
        return n**2 + 2*(m-n) -1
    
    def beta(self, n, m):
        assert (1 <= m) and (m < n) and (n <= self.N)
        return n**2 + 2*(m-n)
    
    def gamma(self, n):
        assert (1 <= n) and (n <= self.N)
        return n**2 - 1

    def generate_generators(self):
        generators = {}
        for n in range(1, self.N+1):
            for m in range(1, self.N+1):
                if m < n:
                    matrix = np.zeros((self.N, self.N))
                    matrix[m-1,n-1] = 1
                    matrix[n-1,m-1] = 1
                    generators[self.alpha(n,m)] = matrix/2

                    matrix = np.zeros((self.N, self.N))
                    matrix[m-1,n-1] = 1
                    matrix[n-1,m-1] = -1
                    generators[self.beta(n,m)] = -1j*matrix/2
            
            if n > 1:
                matrix = np.zeros((self.N, self.N))
                matrix[n-1,n-1] = (1 - n)
                for l in range(1, n):
                    matrix[l-1,l-1] = 1
                generators[self.gamma(n)] = matrix / np.sqrt(2*n*(n-1))

        return generators

    def generate_structure_constants(self):

        structure_constants = {}
        
        '''
        for n in range(1, self.N+1):
            for m in range(1, self.N+1):
                if m < n:
                    print('alpha = %i, beta = %i' %(self.alpha(n,m), self.beta(n,m)))
            print('gamma = %i' % self.gamma(n))
        '''

        for n in range(1, self.N+1):
            for m in range(1, self.N+1):
                for k in range(1, self.N+1):
                    
                    try:
                        structure_constants[(
                            self.alpha(n,m), 
                            self.alpha(k,n), 
                            self.beta(k,m)
                            )] = 1/2
                    except:
                        pass
                    
                    try:
                        structure_constants[(
                            self.alpha(n,m), 
                            self.alpha(n,k), 
                            self.beta(k,m)
                            )] = 1/2
                    except:
                        pass
                    
                    try:
                        structure_constants[(
                            self.alpha(n,m), 
                            self.alpha(k,m), 
                            self.beta(k,n)
                            )] = 1/2
                    except:
                        pass

                    try:
                        structure_constants[(
                            self.beta(n,m), 
                            self.beta(k,m), 
                            self.beta(k,n)
                            )] = 1/2
                    except:
                        pass
                
                    try:
                        structure_constants[(
                            self.alpha(n,m), 
                            self.beta(n,m), 
                            self.gamma(m)
                            )] = - np.sqrt( (m-1)/(2*m) )
                    except:
                        pass

                    try:
                        structure_constants[(
                            self.alpha(n,m), 
                            self.beta(n,m), 
                            self.gamma(n)
                            )] = np.sqrt(n / (2*(n-1)) )
                    except:
                        pass

                    if m < k and k < n:
                        try:
                            structure_constants[(
                                self.alpha(n,m), 
                                self.beta(n,m), 
                                self.gamma(k)
                                )] = np.sqrt(1 / (2*k*(k-1)) )
                        except:
                            pass

        for key, value in list(structure_constants.items()):
            if value == 0:
                structure_constants.pop(key)
            else:
                i, j, k = key
                structure_constants[(i,k,j)] = - value
                structure_constants[(j,i,k)] = - value
                structure_constants[(j,k,i)] = value
                structure_constants[(k,i,j)] = value
                structure_constants[(k,j,i)] = - value
    
        return structure_constants
    
    def check(self):
        for i in range(1, self.N**2-1):
            for j in range(i+1, self.N**2):

                term1 = (np.dot(
                    self.generators[i], 
                    self.generators[j]
                    ) - 
                np.dot(
                    self.generators[j], 
                    self.generators[i]
                    ))

                term2 = sum(
                    1j*self.structure_constants.get((i,j,k), 0)
                    * self.generators[k] 
                    for k in range(1, self.N**2))

                if not np.allclose(term1, term2):
                    print(i,j)
                    print(term1)
                    print(term2)
                    raise AssertionError

In [12]:
number_of_matrices = 3
gauge_group_degree = 2 # the N in SU(N)
number_of_generators = gauge_group_degree**2 - 1 #understand the diff b/w U(N) and SU(N)

bits_per_oscillator = 2
number_states_per_oscillator = 2**bits_per_oscillator

number_qubits = (
    number_of_matrices 
    * number_of_generators 
    * bits_per_oscillator
)

print(f'number_qubits {number_qubits}, Hilbert space dimension {2**number_qubits}')

number_qubits 18, Hilbert space dimension 262144


In [15]:
su_group = SpecialUnitaryGroup(gauge_group_degree)
su_group.structure_constants

{(1, 2, 3): 1.0,
 (1, 3, 2): -1.0,
 (2, 1, 3): -1.0,
 (2, 3, 1): 1.0,
 (3, 1, 2): 1.0,
 (3, 2, 1): -1.0}

In [16]:
index_map = {}
counter = 0
for i in range(number_of_matrices):
    for a in range(number_of_generators):
        index_map[(i,a)] = counter
        counter += bits_per_oscillator
index_map_inverse = dict(map(reversed, index_map.items()))
index_map

{(0, 0): 0,
 (0, 1): 2,
 (0, 2): 4,
 (1, 0): 6,
 (1, 1): 8,
 (1, 2): 10,
 (2, 0): 12,
 (2, 1): 14,
 (2, 2): 16}

In [17]:
single_qubit_state_map = {
    '00':0.5*(sp.symbols("I") + sp.symbols('Z')),
    '01':0.5*(sp.symbols("X") + 1j*sp.symbols('Y')),
    '10':0.5*(sp.symbols("X") - 1j*sp.symbols('Y')),
    '11':0.5*(sp.symbols("I") - sp.symbols('Z'))
    }

In [18]:
def annihilation_operator(i, A):
    qubit_indx0 = index_map[(i,A)]
    operator = 0
    for j in range(number_states_per_oscillator-1):
        bra_bitstring = f'{j:0{bits_per_oscillator}b}'
        ket_bitstring = f'{j+1:0{bits_per_oscillator}b}'
        for i in range(bits_per_oscillator):
            new_term = single_qubit_state_map[bra_bitstring[i] + ket_bitstring[i]]
            new_term = new_term.subs('I', f'I{qubit_indx0+i}')
            new_term = new_term.subs('X', f'X{qubit_indx0+i}')
            new_term = new_term.subs('Y', f'Y{qubit_indx0+i}')
            new_term = new_term.subs('Z', f'Z{qubit_indx0+i}')
            operator += new_term
    return operator


def creation_operator(i, A):
    qubit_indx0 = index_map[(i,A)]
    operator = 0
    for j in range(number_states_per_oscillator-1):
        bra_bitstring = f'{j+1:0{bits_per_oscillator}b}'
        ket_bitstring = f'{j:0{bits_per_oscillator}b}'
        for i in range(bits_per_oscillator):
            new_term = single_qubit_state_map[bra_bitstring[i] + ket_bitstring[i]]
            new_term = new_term.subs('I', f'I{qubit_indx0+i}')
            new_term = new_term.subs('X', f'X{qubit_indx0+i}')
            new_term = new_term.subs('Y', f'Y{qubit_indx0+i}')
            new_term = new_term.subs('Z', f'Z{qubit_indx0+i}')
            operator += new_term
    return operator

In [19]:
string_to_Pauli_map = {'I':opflow.I, 'X':opflow.X, 'Y':opflow.Y, 'Z':opflow.Z}

In [20]:
def sympy_Pauli_to_qiskit_Pauli(operator_string):
    '''CAREFUL WITH ENDIAN CONVENTION'''
    pauli_string = operator_string[0]
    qubit_number = int(operator_string[1:])

    #print(pauli_string, qubit_number)

    if qubit_number == 0:
        new_operator = string_to_Pauli_map[pauli_string]
    else:
        new_operator = opflow.I
    
    for i in range(1, number_qubits):
        if i != qubit_number:
            new_operator = new_operator ^ opflow.I
        else:
            new_operator = new_operator ^ string_to_Pauli_map[pauli_string]
    return new_operator.reduce()

In [21]:
def sympy_operator_to_qiskit_operator(sympy_operator):
    operator = 0 #sympy_Pauli_to_qiskit_Pauli(f'I{number_qubits}')
    for arg in sympy_operator.args:
        #print(arg)
        if len(arg.args) == 2:
            coeff = complex(arg.args[0])
        elif len(arg.args) == 3:
            coeff = complex(sp.prod(arg.args[0:2]))
        else:
            raise ValueError
        #print(coeff, sympy_Pauli_to_qiskit_Pauli(str(arg.args[-1])))
        operator += coeff * sympy_Pauli_to_qiskit_Pauli(str(arg.args[-1]))
    return operator

In [22]:
def position_operator(i, A, sqrt_nu):
    creation = sympy_operator_to_qiskit_operator(creation_operator(i, A))
    annihilation = sympy_operator_to_qiskit_operator(annihilation_operator(i, A))
    coeff = np.sqrt(1/2) / sqrt_nu
    operator = 1 * (creation + annihilation)
    return operator.reduce()


def momentum_operator(i, A, sqrt_nu):
    creation = sympy_operator_to_qiskit_operator(creation_operator(i, A))
    annihilation = sympy_operator_to_qiskit_operator(annihilation_operator(i, A))
    coeff = -1j * sqrt_nu * np.sqrt(1/2)
    operator = coeff * (creation - annihilation)
    return operator.reduce()


def hamiltonian_bosonic_free(sqrt_nu):
    operator = 0 #0*sympy_Pauli_to_qiskit_Pauli(f'I{number_qubits}')
    nu_squared = sqrt_nu * sqrt_nu * sqrt_nu * sqrt_nu
    for i in range(number_of_matrices):
        for a in range(number_of_generators):
            operator += 0.5 * (
                momentum_operator(i, a, sqrt_nu)**2 
                + nu_squared * position_operator(i, a, sqrt_nu)**2
                )
    return operator.reduce()

In [23]:
sqrt_nu = Parameter('sqrt_nu')

In [24]:
H = hamiltonian_bosonic_free(sqrt_nu)
H[0]

PauliSumOp(SparsePauliOp(['IIIIIIIIIIIIIIIIII', 'YYIIIIIIIIIIIIIIII'],
              coeffs=[-1.+0.j, -1.+0.j]), coeff=-0.5*sqrt_nu**2)

In [25]:
for i in range(len(H)):
    print(H[i])

-0.5*sqrt_nu**2 * (
  -1.0 * IIIIIIIIIIIIIIIIII
  - 1.0 * YYIIIIIIIIIIIIIIII
)
1.0*sqrt_nu**4 * (
  7.0 * IIIIIIIIIIIIIIIIII
  + 2.0 * XIIIIIIIIIIIIIIIII
  + 3.0 * XXIIIIIIIIIIIIIIII
  + 6.0 * IXIIIIIIIIIIIIIIII
)
-0.5*sqrt_nu**2 * (
  -1.0 * IIIIIIIIIIIIIIIIII
  - 1.0 * IIYYIIIIIIIIIIIIII
)
1.0*sqrt_nu**4 * (
  7.0 * IIIIIIIIIIIIIIIIII
  + 2.0 * IIXIIIIIIIIIIIIIII
  + 3.0 * IIXXIIIIIIIIIIIIII
  + 6.0 * IIIXIIIIIIIIIIIIII
)
-0.5*sqrt_nu**2 * (
  -1.0 * IIIIIIIIIIIIIIIIII
  - 1.0 * IIIIYYIIIIIIIIIIII
)
1.0*sqrt_nu**4 * (
  7.0 * IIIIIIIIIIIIIIIIII
  + 2.0 * IIIIXIIIIIIIIIIIII
  + 3.0 * IIIIXXIIIIIIIIIIII
  + 6.0 * IIIIIXIIIIIIIIIIII
)
-0.5*sqrt_nu**2 * (
  -1.0 * IIIIIIIIIIIIIIIIII
  - 1.0 * IIIIIIYYIIIIIIIIII
)
1.0*sqrt_nu**4 * (
  7.0 * IIIIIIIIIIIIIIIIII
  + 2.0 * IIIIIIXIIIIIIIIIII
  + 3.0 * IIIIIIXXIIIIIIIIII
  + 6.0 * IIIIIIIXIIIIIIIIII
)
-0.5*sqrt_nu**2 * (
  -1.0 * IIIIIIIIIIIIIIIIII
  - 1.0 * IIIIIIIIYYIIIIIIII
)
1.0*sqrt_nu**4 * (
  7.0 * IIIIIIIIIIIIIIIIII
  + 2.0 * IIIIIIIIX

In [26]:
weights = [0]*len(H)
for i in range(len(H)):
    terms = H[i].primitive.to_list()
    for term in terms:
        for key, value in Counter(term[0]).items():
            if key != "I":
                if value == 3:
                    print(i, H[i].primitive.to_list())
                    break
                weights[i] = max(weights[i], value)
weights

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [ ]:
annihilation_operators = list(sp.symbols(f'adagger00:{number_of_matrices}:{number_of_generators}'))
creation_operators = list(sp.symbols(f'a00:{number_of_matrices}:{number_of_generators}'))

counter = 0
for i in range(number_of_matrices):
    for a in range(number_of_generators):
        block_index = index_map[(i,a)]
        expr = sum(
            sp.sqrt(j+1) * sp.symbols(
            f'bra{|{index_map[(i,a,j+1)]}><{index_map[(i,a,j)]}|'
            ) for j in range(0, number_states_per_oscillator-1))
        annihilation_operators[counter] = annihilation_operators[counter].subs(f'adagger{i}{a}', expr)

        expr = sum(
            sp.sqrt(j+1) * sp.symbols(
            f'|{index_map[(i,a,j)]}><{index_map[(i,a,j+1)]}|'
            ) for j in range(0, number_states_per_oscillator-1))
        creation_operators[counter] = creation_operators[counter].subs(f'a{i}{a}', expr)

        counter += 1


In [ ]:
nu = sp.symbols('nu')

position_operator = sp.MatrixSymbol('positionOperator', number_of_matrices, number_of_generators)
momentum_operator = sp.MatrixSymbol('momentumOperator', number_of_matrices, number_of_generators)
#creation_operator = sp.MatrixSymbol('creationOperator', number_of_matrices, number_of_generators)
#annihilation_operator = sp.MatrixSymbol('annihilationOperator', number_of_matrices, number_of_generators)

#position_operator = sp.sqrt(1/(2 * nu)) * (creation_operator + annihilation_operator)
#momentum_operator = -1j * sp.sqrt(nu/(2)) * (creation_operator - annihilation_operator)

In [ ]:
creation_operators

In [ ]:
expr

In [ ]:
expr.coeff('bra45ket46', 1)

In [ ]:
index_map_inverse[45]

In [ ]:
index_map_inverse

In [ ]:
bits_per_oscillator